In [1]:
import mlflow
from mlflow.tracking import MlflowClient

# Set this to match what your MLflow UI is using (adjust if needed)
mlflow.set_tracking_uri("http://localhost:5000")  # assuming UI is served here

client = MlflowClient()

experiments = client.search_experiments()

for exp in experiments:
    print(f"📁 {exp.experiment_id} — {exp.name}")

/home/mlops/fashion/venv/lib/python3.12/site-packages/mlflow/utils/requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


📁 6 — housing-DecisionTree
📁 5 — Housing-LinearRegression
📁 4 — Iris-Models
📁 3 — FashionMNIST-MLP
📁 2 — FashionMNIST-CNN
📁 0 — Default


In [2]:
import mlflow
from mlflow.tracking import MlflowClient

# Initialize
client = MlflowClient()
experiment_name = "Iris-Models"

# Get experiment by name
experiment = client.get_experiment_by_name(experiment_name)
if not experiment:
    raise ValueError(f"❌ Experiment '{experiment_name}' not found.")

# Get all runs
runs = client.search_runs(experiment.experiment_id)
iris_runs = []

for run in runs:
    metrics = run.data.metrics
    if "accuracy" in metrics:
        iris_runs.append({
            "run_id": run.info.run_id,
            "accuracy": metrics["accuracy"],
            "model_name": run.data.tags.get("mlflow.runName", "Unnamed"),
        })

if not iris_runs:
    raise ValueError("❌ No valid runs with 'accuracy' found!")

# Select the best run based on accuracy
best_iris_run = max(iris_runs, key=lambda x: x["accuracy"])
best_run_id = best_iris_run["run_id"]
print(f"✅ Best Iris model: {best_iris_run['model_name']} (Accuracy: {best_iris_run['accuracy']:.4f})")


✅ Best Iris model: LogisticRegression_Iris (Accuracy: 0.9667)


In [3]:
# Register and Promote
model_name = "IrisBestModel"
artifact_path = "model"  # Change if your model was logged under another artifact name
model_uri = f"runs:/{best_run_id}/{artifact_path}"

# Register model
model_version = mlflow.register_model(model_uri=model_uri, name=model_name)

# Promote to Production
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Production",
    archive_existing_versions=True
)

print(f"🚀 Model version {model_version.version} promoted to 'Production'")


Registered model 'IrisBestModel' already exists. Creating a new version of this model...
2025/07/27 14:30:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IrisBestModel, version 3


🚀 Model version 3 promoted to 'Production'


Created version '3' of model 'IrisBestModel'.
/tmp/ipykernel_2176874/1616656766.py:10: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [4]:
import mlflow
import pandas as pd

# Optional: set tracking URI if not already default
mlflow.set_tracking_uri("http://localhost:5000")

# Load the Production model
model = mlflow.pyfunc.load_model("models:/IrisBestModel/Production")

# Class label mapping
class_map = {
    0: "Iris-setosa",
    1: "Iris-versicolor",
    2: "Iris-virginica"
}

# === Sample Input 1 ===
X_sample1 = pd.DataFrame([[5.1, 3.5, 1.4, 0.2]], columns=[
    "sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"
])
pred1 = model.predict(X_sample1)
print("Predicted class for Sample 1:", class_map[int(pred1[0])])

# === Sample Input 2 ===
X_sample2 = pd.DataFrame([[6.9, 3.1, 5.4, 2.1]], columns=[
    "sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"
])
pred2 = model.predict(X_sample2)
print("Predicted class for Sample 2:", class_map[int(pred2[0])])


/home/mlops/fashion/venv/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


Predicted class for Sample 1: Iris-setosa
Predicted class for Sample 2: Iris-virginica


In [5]:
mlflow.pyfunc.get_model_dependencies("models:/IrisBestModel/Production")


/home/mlops/fashion/venv/lib/python3.12/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.1/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])
2025/07/27 15:21:22 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /home/mlops/mlruns/4/5c6730a6cc254b39a5f53d08b00359db/artifacts/model/requirements.txt'.


'/home/mlops/mlruns/4/5c6730a6cc254b39a5f53d08b00359db/artifacts/model/requirements.txt'